## Dependencies

In [1]:
import warnings
from tensorflow_hub import KerasLayer
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling1D, SpatialDropout1D, Concatenate
from googleqa_utilityscript import *
from googleqa_map_utilityscript import *
import bert_tokenization as tokenization


SEED = 0
seed_everything(SEED)
warnings.filterwarnings("ignore")

## Load data

In [2]:
model_path = '/kaggle/input/114-googleq-a-train-bert-base-uncased-seq-slicesv2/model.h5'
BERT_PATH = '/kaggle/input/tf-hub-bert-base/bert_base_uncased'
VOCAB_PATH = BERT_PATH + '/assets/vocab.txt'

test = pd.read_csv('/kaggle/input/google-quest-challenge/test.csv')

print('Test samples: %s' % len(test))
display(test.head())

Test samples: 476


,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host
0,39,Will leaving corpses lying around upset my pri...,I see questions/information online about how t...,Dylan,https://gaming.stackexchange.com/users/64471,There is no consequence for leaving corpses an...,Nelson868,https://gaming.stackexchange.com/users/97324,http://gaming.stackexchange.com/questions/1979...,CULTURE,gaming.stackexchange.com
1,46,Url link to feature image in the portfolio,I am new to Wordpress. i have issue with Featu...,Anu,https://wordpress.stackexchange.com/users/72927,I think it is possible with custom fields.\n\n...,Irina,https://wordpress.stackexchange.com/users/27233,http://wordpress.stackexchange.com/questions/1...,TECHNOLOGY,wordpress.stackexchange.com
2,70,"Is accuracy, recoil or bullet spread affected ...","To experiment I started a bot game, toggled in...",Konsta,https://gaming.stackexchange.com/users/37545,You do not have armour in the screenshots. Thi...,Damon Smithies,https://gaming.stackexchange.com/users/70641,http://gaming.stackexchange.com/questions/2154...,CULTURE,gaming.stackexchange.com
3,132,Suddenly got an I/O error from my external HDD,I have used my Raspberry Pi as a torrent-serve...,robbannn,https://raspberrypi.stackexchange.com/users/17341,Your Western Digital hard drive is disappearin...,HeatfanJohn,https://raspberrypi.stackexchange.com/users/1311,http://raspberrypi.stackexchange.com/questions...,TECHNOLOGY,raspberrypi.stackexchange.com
4,200,Passenger Name - Flight Booking Passenger only...,I have bought Delhi-London return flights for ...,Amit,https://travel.stackexchange.com/users/29089,I called two persons who work for Saudia (tick...,Nean Der Thal,https://travel.stackexchange.com/users/10051,http://travel.stackexchange.com/questions/4704...,CULTURE,travel.stackexchange.com


In [3]:
question_target_cols = ['question_asker_intent_understanding','question_body_critical', 'question_conversational', 
                        'question_expect_short_answer', 'question_fact_seeking', 'question_has_commonly_accepted_answer',
                        'question_interestingness_others', 'question_interestingness_self', 'question_multi_intent', 
                        'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice',
                        'question_type_compare', 'question_type_consequence', 'question_type_definition', 
                        'question_type_entity', 'question_type_instructions', 'question_type_procedure',
                        'question_type_reason_explanation', 'question_type_spelling', 'question_well_written']
answer_target_cols = ['answer_helpful', 'answer_level_of_information', 'answer_plausible', 'answer_relevance',
                      'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure', 
                      'answer_type_reason_explanation', 'answer_well_written']
target_cols = question_target_cols + answer_target_cols

## Pre-process data

In [4]:
text_features = ['question_title', 'question_body', 'answer']
    
for feature in text_features:
    # Lower
    test[feature] = test[feature].apply(lambda x: x.lower())
    # Map misspellings
    test[feature] = test[feature].apply(lambda x: map_misspellings(x))
    # Map contractions
    test[feature] = test[feature].apply(lambda x: map_contraction(x))
    # Trim text
    test[feature] = test[feature].apply(lambda x: x.strip())

# Model parameters

In [5]:
N_CLASS = len(target_cols)
MAX_SEQUENCE_LENGTH = 512

## Test set

In [6]:
tokenizer = tokenization.FullTokenizer(VOCAB_PATH, do_lower_case=True)

# Test features
X_test = compute_input_arays(test, text_features, tokenizer, MAX_SEQUENCE_LENGTH)

# Model

In [7]:
input_word_ids = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_word_ids')
input_masks = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_masks')
segment_ids = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='segment_ids')

bert_layer = KerasLayer(BERT_PATH, trainable=True)
pooled_output, sequence_output = bert_layer([input_word_ids, input_masks, segment_ids])

t = GlobalAveragePooling1D()(sequence_output[:,:32])
t = Dropout(0.2)(t)

q = GlobalAveragePooling1D()(sequence_output[:,32:272])
q = Dropout(0.2)(q)

question = Concatenate()([t, q])
question = Dense(21 , kernel_initializer='glorot_uniform', activation="sigmoid")(question)

a = GlobalAveragePooling1D()(sequence_output[:,272:])
a = Dropout(0.2)(a)
a = Dense(9, kernel_initializer='glorot_uniform', activation="sigmoid")(a)

output = Concatenate(name="output")([question, a])

model = Model(inputs=[input_word_ids, input_masks, segment_ids], outputs=output)

model.summary()

model.load_weights(model_path)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 512)]        0                                            
__________________________________________________________________________________________________
input_masks (InputLayer)        [(None, 512)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 512)]        0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 768), (None, 109482241   input_word_ids[0][0]             
                                                                 input_masks[0][0]            

# Make predictions

In [8]:
Y_test = model.predict(X_test)

In [9]:
submission = pd.read_csv('/kaggle/input/google-quest-challenge/sample_submission.csv')
submission[target_cols] = Y_test
submission.to_csv("submission.csv", index=False)
display(submission.head())
display(submission.describe())

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.950032,0.699860,0.413357,0.373360,0.506507,0.386052,0.656133,0.624760,0.621221,...,0.949498,0.956421,0.576656,0.978202,0.973051,0.846995,0.022211,0.050035,0.901865,0.924380
1,46,0.873603,0.435373,0.008016,0.713400,0.787217,0.888723,0.546658,0.529200,0.092340,...,0.664863,0.972197,0.643773,0.977454,0.980125,0.897128,0.947560,0.151027,0.057080,0.898421
2,70,0.895755,0.523496,0.059155,0.633642,0.755933,0.848452,0.605806,0.549028,0.192711,...,0.857639,0.954895,0.578760,0.970593,0.969508,0.857757,0.082417,0.067726,0.886554,0.925947
3,132,0.889313,0.380248,0.009511,0.659805,0.674026,0.877537,0.507265,0.500591,0.062903,...,0.745855,0.963504,0.646577,0.964245,0.980531,0.886036,0.844855,0.186565,0.230002,0.881543
4,200,0.926595,0.554131,0.046043,0.757462,0.693863,0.830533,0.610183,0.608693,0.147053,...,0.710687,0.940687,0.685800,0.964386,0.967561,0.866790,0.194188,0.135696,0.751425,0.900629


,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
count,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,...,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000
mean,5029.186975,0.897208,0.552736,0.055683,0.671470,0.756151,0.795116,0.570353,0.511523,0.167812,...,0.810051,0.947660,0.643729,0.966196,0.970076,0.874824,0.561731,0.164965,0.460703,0.899347
std,2812.670060,0.041026,0.119739,0.093324,0.112663,0.096616,0.143994,0.056827,0.078943,0.142811,...,0.084684,0.023580,0.045735,0.010496,0.012632,0.038863,0.329668,0.058074,0.288558,0.023555
min,39.000000,0.767653,0.320180,0.003890,0.245915,0.250918,0.194213,0.456371,0.384093,0.011043,...,0.599401,0.830932,0.494386,0.916165,0.918233,0.696177,0.007466,0.027334,0.019526,0.815880
25%,2572.000000,0.869128,0.453014,0.011992,0.607107,0.719671,0.755833,0.527280,0.455180,0.062854,...,0.741969,0.939283,0.614227,0.961629,0.964425,0.859956,0.241180,0.126111,0.203530,0.883312
50%,5093.000000,0.897839,0.549650,0.019367,0.670056,0.765281,0.845647,0.560170,0.488335,0.113110,...,0.814160,0.954062,0.647203,0.967812,0.973612,0.882408,0.669858,0.171628,0.434778,0.900259
75%,7482.000000,0.930247,0.645294,0.052095,0.735328,0.813324,0.887400,0.605863,0.548473,0.235073,...,0.886323,0.963086,0.674389,0.973046,0.979344,0.901987,0.860114,0.206862,0.703854,0.917239
max,9640.000000,0.973624,0.828714,0.743814,0.958111,0.977558,0.962701,0.734610,0.749239,0.746979,...,0.949498,0.986175,0.778078,0.989226,0.990456,0.955655,0.963488,0.300946,0.981473,0.948430
